In [ ]:
import biopathopt
import cobra
import pandas as pd
import plotly.express as px

# Enzyme Contrained Model

In this notebook, we investigate how to generate an enzyme constrained model

In [ ]:
ecm = biopathopt.EnzymeConstrainedModel(
    path_to_model='models/iML1515R.xml', 
    taxonomy_id=83333,
    use_progressbar=True,
)

In [ ]:
ecm.enzyme_enrich_model(use_progressbar=True)

In [ ]:
updated_reacs = ecm.optimize_ec_model(
    target_growth_rate_h=0.66,
    model_objective='BIOMASS_Ec_iML1515_core_75p37M',
    carbon_exchange_reaction='EX_glc__D_e',
    use_progressbar=True,
    maximum_optimization_rounds=50,
)

In [ ]:
ecm.save_model(file_path='models/iML1515R_parsed_enriched_optimized.json.gzip')

In [ ]:
ec_model = ecm.return_ec_model(
    substrate_exchange_reaction_id='EX_glc__D_e',
)

## kcat and enzyme mass distributions

In [ ]:
ecm = biopathopt.EnzymeConstrainedModel(
    path_to_model='models/iML1515R_parsed_enriched_optimized.json.gzip', 
)

In [ ]:
ec_model = ecm.return_ec_model(
    substrate_exchange_reaction_id='EX_glc__D_e',
)

In [ ]:
import plotly.io as pio
pio.renderers.default = 'notebook'

In [ ]:
kcat = [r.annotation.get('kcat') for r in ec_model.reactions]
fig = biopathopt.plots.cdf(kcat, x_label='kcat(1/s)')
pio.show(fig)

In [ ]:
mw = [r.annotation.get('mw') for r in ec_model.reactions]
fig = biopathopt.plots.cdf(mw, x_label='mass(kDa)')
pio.show(fig)

## Compare Experimental to Measured 

In [ ]:
growth_exp = pd.read_csv('data/growth_exp.csv', index_col=0)
substrates = list(growth_exp.index)
norm_model = cobra.io.read_sbml_model('models/iML1515R.xml')
#growth = pd.DataFrame()
for substrate in substrates:
    with ec_model as growth_model: 
        growth_model.reactions.get_by_id('EX_glc__D_e_reverse').bounds =(0.0, 0.0) 
        growth_model.reactions.get_by_id(substrate).bounds = (-10, 0.0)
        try:
            pfba_solution = cobra.flux_analysis.pfba(growth_model)
            growth_exp.loc[substrate, 'ECMpy_flux'] = pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M']
        except cobra.exceptions.Infeasible as e:
            print(f'Substrate {substrate} is infeasible  in ec model')
            pass
        #growth_exp.loc[substrate, 'sub_flux'] = pfba_solution.fluxes[substrate]
        
for substrate in substrates:
    with norm_model as growth_model: 
        growth_model.reactions.get_by_id('EX_glc__D_e').bounds =(0.0, 0.0) 
        growth_model.reactions.get_by_id(substrate).bounds = (-10, 0.0)
        pfba_solution = cobra.flux_analysis.pfba(growth_model)
        growth_exp.loc[substrate, 'iML1515_flux'] = pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M']
        #growth_exp.loc[substrate, 'sub_flux'] = pfba_solution.fluxes[substrate]      

In [ ]:
fig = biopathopt.plots.predicted_v_sim(
    df = growth_exp,
    measured_col='EXP',
    sim_col='iML1515_flux',
    name_col='substrate',
    title='Original Model Simulated vs Measured'
)
pio.show(fig)

In [ ]:
fig = biopathopt.plots.predicted_v_sim(
    df = growth_exp,
    measured_col='EXP',
    sim_col='ECMpy_flux',
    name_col='substrate',
    title='Enzyme Contrained Model Simulated vs Measured'
)
pio.show(fig)

In [ ]:
a = pd.DataFrame(abs(growth_exp["EXP"]-growth_exp["iML1515_flux"])/growth_exp["EXP"], columns=['Estimated Error'])
a['type'] = 'Original Model'
b = pd.DataFrame(abs(growth_exp["EXP"]-growth_exp["ECMpy_flux"])/growth_exp["EXP"], columns=['Estimated Error'])
b['type'] = 'Enzyme Contrained Model'
df = pd.concat([a,b])
fig = px.box(df, x='type', y='Estimated Error', width=800, height=500)
pio.show(fig)